# 1. Set-up

In [1]:
# a temporary solution, run to set the current path as '../Fair-RL'
%cd ..

/Users/xiaohui/Documents/Code/Fair-RL


/Users/xiaohui/Documents/Code/Fair-RL/venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
# used to auto-reload imported module after making modification inside .py files
%load_ext autoreload
%autoreload 2

## 1.1 imports

In [3]:
from experiments.configs.base import params
from env.mrp_env_rccc import MachineReplacement
from solver.ggf_dlp import build_dlp, extract_dlp, solve_dlp
from utils.common import MDP4LP
from utils.mrp import MRPData
from utils.policy import calculate_state_value
import pandas as pd
import numpy as np
from utils.encoding import state_vector_to_int_index, state_int_index_to_vector
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
# set parameters
params.num_actions = 2
params.num_states = 3
params.num_groups = 5

mrp_data = MRPData(
    num_groups=params.num_groups,
    num_states=params.num_states,
    num_actions=params.num_actions,
    prob_remain=params.prob_remain,
    weight_type=params.weight_type,
    cost_types_operation=params.cost_type_operation,
    cost_types_replace=params.cost_type_replace,
)

mdp = MDP4LP(
    num_states=mrp_data.num_global_states,
    num_actions=mrp_data.num_global_actions,
    num_groups=mrp_data.num_groups,
    transition=mrp_data.global_transitions,
    costs=mrp_data.global_costs,
    discount=params.gamma,
    weights=mrp_data.weights,
    minimize=True,
    encoding_int=False,
    base_num_states=params.num_states,
)

In [39]:
params.weight_type = 'exponential3'

In [8]:
# set parameters
params.num_actions = 2
params.num_states = 3
params.num_groups = 2

mrp_data = MRPData(
    num_groups=params.num_groups,
    num_states=params.num_states,
    num_actions=params.num_actions,
    prob_remain=params.prob_remain,
    weight_type=params.weight_type,
    cost_types_operation="quadratic",
    cost_types_replace="zero",
)

mdp = MDP4LP(
    num_states=mrp_data.num_global_states,
    num_actions=mrp_data.num_global_actions,
    num_groups=mrp_data.num_groups,
    transition=mrp_data.global_transitions,
    costs=mrp_data.global_costs,
    discount=params.gamma,
    weights=mrp_data.weights,
    minimize=True,
    encoding_int=False,
    base_num_states=params.num_states,
)

In [9]:
# calculate LP values
uniform_dist = [1 / mrp_data.num_global_states for i in range(mrp_data.num_global_states)]
model = build_dlp(mdp=mdp, initial_mu=uniform_dist)
# Solve the GGF model
_, model, _ = solve_dlp(model=model)
results = extract_dlp(model=model, print_results=True)

Initial distribution:  [0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111]
Solver solving time: 0.0538
Number of solutions found: 1
Proportion of stochastic policy: 20.0%
Policy:                        Var X:            Costs - Group 1 | Group 2                        
        0     1     2     0       1        2           0    1     2        0     1    2
(0, 0)  0     0     1     0       0  11.4075        0.00  0.0  0.00     0.00  0.00  0.0
(0, 1)  0     0     1     0       0   3.4683        0.00  0.0  0.00     0.25  0.25  0.0
(0, 2)  0     0     1     0       0   0.2444        0.00  0.0  0.00     1.00  1.00  0.0
(1, 0)  0     1     0     0  3.1981        0        0.25  0.0  0.25     0.00  0.00  0.0
(1, 1)  0  0.72  0.28     0  0.7282    0.288        0.25  0.0  0.25     0.25  0.25  0.0
(1, 2)  0     0     1     0       0   0.1444        0.25  0.0  0.25     1

In [41]:
# calculate LP values
uniform_dist = [1 / mrp_data.num_global_states for i in range(mrp_data.num_global_states)]
model = build_dlp(mdp=mdp, initial_mu=uniform_dist)
# Solve the GGF model
_, model, _ = solve_dlp(model=model)
results = extract_dlp(model=model, print_results=True)

Initial distribution:  [0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111, 0.1111111111111111]
Solver solving time: 0.0085
Number of solutions found: 1
Proportion of stochastic policy: 20.0%
Policy:                        Var X:            Costs - Group 1 | Group 2                        
        0    1    2          0       1       2           0    1     2        0     1    2
(0, 0)  1    0    0     3.2918       0       0        0.00  1.0  0.00     0.00  0.00  1.0
(0, 1)  1    0    0     3.9055       0       0        0.00  1.0  0.00     0.17  0.17  1.0
(0, 2)  0    0    1          0       0  0.9698        0.00  1.0  0.00     0.67  0.67  1.0
(1, 0)  1    0    0     3.9055       0       0        0.17  1.0  0.17     0.00  0.00  1.0
(1, 1)  1    0    0     4.6161       0       0        0.17  1.0  0.17     0.17  0.17  1.0
(1, 2)  0    0    1          0       0  1.0274        0.17  1

In [6]:
from utils.count import CountMDP

In [25]:
count_mdp = CountMDP(
    num_groups=2,
    num_states=params.num_states,
    num_actions=params.num_actions
)

In [9]:
from solver.count_dlp import build_count_dlp, extract_count_dlp, solve_count_dlp

In [18]:
# calculate LP values
uniform_dist = [1/len(count_mdp.count_states)] * len(count_mdp.count_states)
model = build_count_dlp(count_mdp=count_mdp, initial_mu=uniform_dist)
# Solve the GGF model
_, model = solve_count_dlp(model=model)
results = extract_count_dlp(model=model, print_results=True)

          [0 0 0] [1 0 0] [0 1 0] [0 0 1]
[5, 0, 0]  0.1638       0       0       0
[4, 1, 0]  1.0568       0       0       0
[4, 0, 1]       0       0       0  0.3492
[3, 2, 0]  2.6876       0       0       0
[3, 1, 1]       0       0       0  1.6306
[3, 0, 2]       0       0       0  0.3071
[2, 3, 0]  3.3807       0       0       0
[2, 2, 1]       0       0       0  2.8057
[2, 1, 2]       0       0       0  0.9371
[2, 0, 3]       0       0       0  0.1396
[1, 4, 0]  2.1667       0       0       0
[1, 3, 1]       0       0       0  2.1393
[1, 2, 2]       0       0       0  0.9136
[1, 1, 3]       0       0       0  0.2349
[1, 0, 4]       0       0       0  0.0343
[0, 5, 0]       0       0  0.3694       0
[0, 4, 1]       0       0       0  0.3901
[0, 3, 2]       0       0       0   0.192
[0, 2, 3]       0       0       0  0.0731
[0, 1, 4]       0       0       0   0.024
[0, 0, 5]       0       0       0  0.0041
          [0 0 0]    [1 0 0]    [0 1 0]    [0 0 1]
[5, 0, 0]       0        

In [20]:
20-4.0899

15.9101

In [22]:
# calculate LP values
uniform_dist = [1/len(count_mdp.count_states)] * len(count_mdp.count_states)
model = build_count_dlp(count_mdp=count_mdp, initial_mu=uniform_dist)
# Solve the GGF model
_, model = solve_count_dlp(model=model)
results = extract_count_dlp(model=model, print_results=True)

          [0 0 0] [1 0 0] [0 1 0] [0 0 1]
[4, 0, 0]  0.4676       0       0       0
[3, 1, 0]   2.329       0       0       0
[3, 0, 1]       0       0       0  0.7023
[2, 2, 0]  4.2973       0       0       0
[2, 1, 1]       0       0       0  2.3595
[2, 0, 2]       0       0       0  0.4047
[1, 3, 0]  3.4851       0       0       0
[1, 2, 1]       0       0       0   2.601
[1, 1, 2]       0       0       0  0.7802
[1, 0, 3]       0       0       0  0.1117
[0, 4, 0]  1.0496       0       0       0
[0, 3, 1]       0       0       0   0.943
[0, 2, 2]       0       0       0  0.3646
[0, 1, 3]       0       0       0  0.0905
[0, 0, 4]       0       0       0  0.0139
          [0 0 0]    [1 0 0]    [0 1 0]    [0 0 1]
[4, 0, 0]       0          1  1000000.0  1000000.0
[3, 1, 0]    0.17       1.17          1  1000000.0
[3, 0, 1]    0.67       1.67  1000000.0          1
[2, 2, 0]    0.33       1.33       1.17  1000000.0
[2, 1, 1]    0.83       1.83       1.67       1.17
[2, 0, 2]    1.33     

In [24]:
# calculate LP values
uniform_dist = [1/len(count_mdp.count_states)] * len(count_mdp.count_states)
model = build_count_dlp(count_mdp=count_mdp, initial_mu=uniform_dist)
# Solve the GGF model
_, model = solve_count_dlp(model=model)
results = extract_count_dlp(model=model, print_results=True)

          [0 0 0] [1 0 0] [0 1 0] [0 0 1]
[3, 0, 0]  1.2698       0       0       0
[2, 1, 0]  4.6165       0       0       0
[2, 0, 1]       0       0       0  1.2647
[1, 2, 0]  5.5497       0       0       0
[1, 1, 1]       0       0       0  2.7453
[1, 0, 2]       0       0       0   0.425
[0, 3, 0]  2.2086       0       0       0
[0, 2, 1]       0       0       0  1.4755
[0, 1, 2]       0       0       0   0.391
[0, 0, 3]       0       0       0  0.0538
          [0 0 0]    [1 0 0]    [0 1 0]    [0 0 1]
[3, 0, 0]       0          1  1000000.0  1000000.0
[2, 1, 0]    0.17       1.17          1  1000000.0
[2, 0, 1]    0.67       1.67  1000000.0          1
[1, 2, 0]    0.33       1.33       1.17  1000000.0
[1, 1, 1]    0.83       1.83       1.67       1.17
[1, 0, 2]    1.33       2.33  1000000.0       1.67
[0, 3, 0]     0.5  1000000.0       1.33  1000000.0
[0, 2, 1]       1  1000000.0       1.83       1.33
[0, 1, 2]     1.5  1000000.0       2.33       1.83
[0, 0, 3]     2.0  1000000.0

In [26]:
# calculate LP values
uniform_dist = [1/len(count_mdp.count_states)] * len(count_mdp.count_states)
model = build_count_dlp(count_mdp=count_mdp, initial_mu=uniform_dist)
# Solve the GGF model
_, model = solve_count_dlp(model=model)
results = extract_count_dlp(model=model, print_results=True)

          [0 0 0] [1 0 0] [0 1 0] [0 0 1]
[2, 0, 0]  3.2918       0       0       0
[1, 1, 0]  7.8111       0       0       0
[1, 0, 1]       0       0       0  1.9396
[0, 2, 0]  4.6161       0       0       0
[0, 1, 1]       0       0       0  2.0549
[0, 0, 2]       0       0       0  0.2865
          [0 0 0]    [1 0 0]    [0 1 0]    [0 0 1]
[2, 0, 0]       0          1  1000000.0  1000000.0
[1, 1, 0]    0.17       1.17          1  1000000.0
[1, 0, 1]    0.67       1.67  1000000.0          1
[0, 2, 0]    0.33  1000000.0       1.17  1000000.0
[0, 1, 1]    0.83  1000000.0       1.67       1.17
[0, 0, 2]    1.33  1000000.0  1000000.0       1.67
Objective:  3.8275
